# Notebook ICD - 16
Libraries

In [2]:
import numpy as np
import pandas as pd

# **SVM desde cero**  
Esta implementación sigue los principios básicos de las Máquinas de Vectores de Soporte (SVM), donde el objetivo principal es encontrar el hiperpunto óptimo que separa las clases. El clasificador se inicializa con tres parámetros clave:

- **Tasa de aprendizaje:** controla cuán rápido se ajusta el modelo durante el entrenamiento.  
- **Parámetro de regularización (lambda):** previene el sobreajuste al equilibrar el margen y los errores.  
- **Número de iteraciones:** establece cuántas veces el algoritmo debe iterar sobre el conjunto de datos para optimizar el hiperpunto.

Dentro de la clase, el método `fit` se utiliza para entrenar el modelo ajustando los pesos (\(w\)) y el sesgo (\(b\)) a través del descenso de gradiente. Durante el entrenamiento, cada punto de datos se clasifica según si satisface la condición del margen, y si no lo hace, tanto los pesos como el sesgo se actualizan en consecuencia.

El método `predict` utiliza los pesos y el sesgo aprendidos para clasificar nuevas instancias calculando el signo de la frontera de decisión.

In [5]:
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        """
        Initialize the SVM with hyperparameters for learning rate, regularization, and number of iterations.
        
        learning_rate: The step size for each iteration of gradient descent.
        lambda_param: Regularization parameter to prevent overfitting.
        n_iters: The number of times the algorithm will iterate over the dataset to find the optimal hyperplane.
        """
        self.learning_rate = learning_rate  # Controls the speed of convergence
        self.lambda_param = lambda_param    # Regularization parameter (controls the margin)
        self.n_iters = n_iters              # Number of training iterations
        self.w = None                       # Weight vector (learned parameters)
        self.b = None                       # Bias term

    def fit(self, X, y):
        """
        Train the SVM model using the training data.
        
        X: The training feature matrix (n_samples, n_features)
        y: The training labels (n_samples,). Labels should be in {-1, 1}.
        
        This method applies gradient descent to optimize the weights (w) and bias (b)
        to maximize the margin between the classes.
        """
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)  # Convert labels to -1 and 1 for SVM

        # Initialize the weight vector and bias term
        self.w = np.zeros(n_features)
        self.b = 0

        # Gradient descent optimization loop
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # If the condition holds, apply a regularization update (no penalty)
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w)
                else:
                    # If the condition fails, apply the update to w and b to penalize the misclassification
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.learning_rate * y_[idx]

    def predict(self, X):
        """
        Predict the labels for the test data.
        
        X: Test feature matrix.
        
        Returns the predicted labels (either -1 or 1).
        """
        # Return the sign of the dot product (linear decision boundary)
        return np.sign(np.dot(X, self.w) - self.b)

# **Ejemplo de implementación**  
Siguiendo la definición de SVM, el conjunto de datos de Clima (Jugar al tenis) se carga en memoria utilizando pandas. Este conjunto de datos incluye varios atributos relacionados con el clima, como perspectiva, temperatura, humedad y viento, que se utilizan para predecir si se puede jugar al tenis. Esta etapa prepara los datos en bruto para un posterior preprocesamiento.

El siguiente paso, el preprocesamiento de datos, es esencial porque SVM requiere entrada numérica. Cada característica categórica en el conjunto de datos (por ejemplo, soleado, caluroso, normal) se asigna a un valor entero correspondiente, lo que permite que el algoritmo procese la información de manera efectiva. La etiqueta objetivo (si se puede jugar al tenis o no) también se convierte en valores numéricos (0 para "no" y 1 para "sí").

Una vez que se completa el preprocesamiento, el conjunto de datos se divide en características y etiquetas. La matriz de características (\(X\)) incluye todas las condiciones climáticas (perspectiva, temperatura, humedad, viento), mientras que el vector de etiquetas (\(y\)) contiene los valores objetivo (jugar al tenis: sí o no). Esta separación permite que el modelo aprenda del conjunto de características mientras predice las etiquetas objetivo correspondientes.

In [10]:
# Load the dataset (assumed to be uploaded or present in the local system)
data = pd.read_csv('weather.nominal.csv')
print(data.head())

# Convert categorical variables into numerical values for KNN
data['outlook'] = data['outlook'].map({'sunny': 0, 'overcast': 1, 'rainy': 2})
data['temperature'] = data['temperature'].map({'hot': 0, 'mild': 1, 'cool': 2})
data['humidity'] = data['humidity'].map({'high': 0, 'normal': 1})
data['windy'] = data['windy'].astype(int)
data['play'] = data['play'].map({'no': -1, 'yes': 1})

# Define X (features) and y (labels)
X = data.drop(columns='play').values  # Features
y = data['play'].values  # Labels

# Show the first transformed data
#print(X)

    outlook temperature humidity  windy play
0     sunny         hot     high  False   no
1     sunny         hot     high   True   no
2  overcast         hot     high  False  yes
3     rainy        mild     high  False  yes
4     rainy        cool   normal  False  yes


A continuación, el clasificador SVM se inicializa con los hiperparámetros especificados, y se ejecuta el proceso de entrenamiento utilizando el método `fit`. El modelo itera sobre el conjunto de datos múltiples veces, ajustando el hiperpunto para maximizar el margen entre las dos clases (sí y no). A través de cada iteración, mejora su capacidad de predicción refinando el vector de pesos y el sesgo para minimizar los errores de clasificación.

In [13]:
# Initialize and Train the SVM Classifier
svm = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
svm.fit(X, y)

# This step initializes the SVM classifier with a specified learning rate, regularization parameter (lambda), and the number of iterations.
# The `fit` method is used to train the classifier using the feature matrix X and the target labels y.
# The model optimizes the weights and bias to create a hyperplane that separates the classes.

Una vez que el modelo está entrenado, se crea una instancia de prueba para simular nuevas condiciones climáticas (soleado, caluroso, humedad normal y ventoso). Esta instancia se representa como un arreglo numérico, donde cada característica está codificada para coincidir con el formato utilizado durante el entrenamiento. Finalmente, se emplea el método `predict` del clasificador para clasificar esta instancia de prueba. Basado en el hiperpunto aprendido, el modelo determina si es un día adecuado para jugar al tenis, y el resultado se muestra como "sí" o "no".

In [18]:
# Create a Test Instance
test_instance = np.array([[0, 0, 1, 1]])  # sunny, hot, normal, TRUE

# This step defines a new test instance that represents a weather condition: sunny, hot, normal humidity, and windy.
# The instance is passed as a NumPy array, where each feature is encoded numerically (sunny = 0, hot = 0, normal = 1, windy = TRUE = 1).

# Make a Prediction
prediction = svm.predict(test_instance)
print(f"Prediction for the test instance: {'yes' if prediction[0] == 1 else 'no'}")

# Finally, the trained SVM classifier makes a prediction on the test instance.
# The `predict` method returns a label: either 1 (yes, play tennis) or -1 (no, don't play tennis).
# The output is printed, interpreting the predicted label in the context of the weather conditions.

Prediction for the test instance: yes


**Implementación en Scikit-learn**  
Las máquinas de vectores de soporte (SVM) son un conjunto de métodos de aprendizaje supervisado utilizados para clasificación, regresión y detección de outliers. Las ventajas de las máquinas de vectores de soporte incluyen:

- Efectivas en espacios de alta dimensionalidad.
- Siguen siendo efectivas en casos donde el número de dimensiones es mayor que el número de muestras.
- Utilizan un subconjunto de puntos de entrenamiento en la función de decisión (llamados vectores de soporte), por lo que también son eficientes en memoria.
- Versátiles: se pueden especificar diferentes funciones de núcleo para la función de decisión. Se proporcionan núcleos comunes, pero también es posible especificar núcleos personalizados.

Las desventajas de las máquinas de vectores de soporte incluyen:

- Si el número de características es mucho mayor que el número de muestras, evitar el sobreajuste al elegir funciones de núcleo y el término de regularización es crucial.
- Las clases SVC, NuSVC y LinearSVC son capaces de realizar clasificación binaria y multiclase en un conjunto de datos. La implementación de C-Support Vector Classification (SVC) se basa en libsvm. El tiempo de ajuste escala al menos de forma cuadrática con el número de muestras y puede ser poco práctico más allá de decenas de miles de muestras. Para conjuntos de datos grandes, considera usar LinearSVC o SGDClassifier en su lugar, posiblemente después de un transformador de Nystroem u otra aproximación de núcleo.

El soporte para múltiples clases se maneja según un esquema de uno contra uno.